In [1]:
import os

os.makedirs('./4.model')
os.makedirs('./5.type_1')
os.makedirs('./5.type_1/0.error')

In [2]:
import pandas as pd

input_ = pd.read_excel('./3.model_1.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)
input_['Make No'] = input_['Make No'].astype(int)
input_['Model No'] = input_['Model No'].astype(int)

input_

,Year,Make No,Make,Make Code,Model No,Model,Model Code
0,2025,1,Acura,58,1,Integra,742
1,2025,1,Acura,58,2,MDX,744
2,2025,2,Chevrolet,47,1,Blazer EV,31147
3,2025,2,Chevrolet,47,2,Silverado 2500 HD,494
4,2025,2,Chevrolet,47,3,Silverado 3500 HD,6580
...,...,...,...,...,...,...,...
48830,1928,3,Plymouth,43,1,Model Q,5604
48831,1925,1,Chevrolet,47,1,Superior,5492
48832,1923,1,Ford,54,1,Model T,5385
48833,1922,1,Ford,54,1,Model T,5385


In [3]:
input_2 = input_.drop_duplicates(['Make'],
                                 keep='first',
                                 ignore_index=True)
input_2 = input_2.sort_values(by=['Make'],
                              ascending=[True],
                              ignore_index=True)

input_2['Make No'] = [_+1 for _ in range(len(input_2))]
input_2['File Make'] = [_.strip().replace('/', '%2F').replace('\\', '%5C') for _ in input_2['Make'].tolist()]

input_2

,Year,Make No,Make,Make Code,Model No,Model,Model Code,File Make
0,1971,1,AC,457,1,Shelby Cobra,21378,AC
1,2001,2,AM General,44,1,Hummer,383,AM General
2,1993,3,AMERTEK,5501416,1,CF4000L,5542122,AMERTEK
3,2008,4,ATC,5501429,1,T815,5601555,ATC
4,2025,5,Acura,58,1,Integra,742,Acura
...,...,...,...,...,...,...,...,...
191,1955,192,Willys,459,1,Aero Ace,21110,Willys
192,2024,193,Winnebago,1373,1,Forza,29887,Winnebago
193,2012,194,Workhorse,608,1,W62,21637,Workhorse
194,2012,195,Workhorse Custom Chassis,1297,1,W42,21636,Workhorse Custom Chassis


In [4]:
from tqdm import tqdm

for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
    output = input_[input_['Make'] == input_2.loc[i, 'Make']].reset_index(drop=True)
    output = output.sort_values(by=['Model', 'Year'],
                                ascending=[True, False],
                                ignore_index=True)
    
    output['Make No'] = input_2.loc[i, 'Make No']
    
    output.loc[0, 'Model No'] = 1
    for j in range(1, len(output)):
        if output.loc[j, 'Model'] == output.loc[j-1, 'Model']:
            output.loc[j, 'Model No'] = output.loc[j-1, 'Model No']
        else:
            output.loc[j, 'Model No'] = output.loc[j-1, 'Model No'] + 1

    output[['Make No',
            'Make',
            'Make Code',
            'Model No',
            'Model',
            'Model Code',
            'Year']].to_excel(f'''./4.model/make_{input_2.loc[i, 'Make No']}.{input_2.loc[i, 'File Make']}.xlsx''', index=False)

print('Done ~')

Progress: 100%|████████████████████████████| 196/196 [00:13<00:00, 14.78it/s]

Done ~
